In [5]:
import pymongo
import operator
import itertools
from pprint import pprint

In [6]:
con = pymongo.MongoClient()
db = con.lacuerda
art = db.artists
ver = db.versions
songs = db.songs

def show_cursor(cursor, limit=10):
    for item in itertools.islice(cursor, limit):
        pprint(item)

### Ver si el album de una canción es el mismo en todas las versiones

In [7]:
cur=ver.aggregate([
        {"$match": {"album": {"$ne": None}}},
        {"$group": {"_id": {"artista": "$artista", "slug": "$slug"},
                    # "versiones": {"$sum": 1},
                    "albums": {"$addToSet": "$album"}}},
        {"$project": {"_id": 1, "albums": 1, "album_count": {"$size": "$albums"}}},
        {"$sort": {"album_count": -1}},
        {"$limit": 10}
    ])
show_cursor(cur)

{'_id': {'slug': 'extranio', 'artista': 'rodriguez'}, 'albums': ['Hasta Luego', 'Palabras Mas Palabras Menos'], 'album_count': 2}
{'_id': {'slug': 'ahi_ahi', 'artista': 'silvestre_dangon'}, 'albums': ['La Fama (2007)', 'La Fama'], 'album_count': 2}
{'_id': {'slug': 'autos_moda_y_rock_and_roll', 'artista': 'moderatto'}, 'albums': ['Tema de la Pelicula cars 2', 'Tema de la Pelicula cars 2 (2011)'], 'album_count': 2}
{'_id': {'slug': 'por_que', 'artista': 'floricienta_y_su_banda'}, 'albums': ['Floricienta y su Banda (2005)', 'Karaoke (2004)'], 'album_count': 2}
{'_id': {'slug': 'que_mas_da', 'artista': 'haash'}, 'albums': ['Hecho Realidad (2014)', 'Ha ash 1f (2014)'], 'album_count': 2}
{'_id': {'slug': 'nunca_aprendimos', 'artista': 'yahir'}, 'albums': ['Elemental (2009)'], 'album_count': 1}
{'_id': {'slug': 'amiga', 'artista': 'yahir'}, 'albums': ['Tributo a Roberto Carlos (2005)'], 'album_count': 1}
{'_id': {'slug': 'en_ti', 'artista': 'yahir'}, 'albums': ['Elemental (2009)'], 'album_co

Conclusión: depende de la versión

### Ver si el compositor de una canción es el mismo en todas las versiones

In [9]:
cur=ver.aggregate([
        {"$match": {"autor": {"$ne": None}}},
        {"$group": {"_id": {"artista": "$artista", "slug": "$slug"},
                    # "versiones": {"$sum": 1},
                    "autores": {"$addToSet": "$autor"}}},
        {"$project": {"_id": 1, "autores": 1, "autor_count": {"$size": "$autores"}}},
        {"$sort": {"autor_count": -1}},
        {"$limit": 10}
    ])
show_cursor(cur)

{'_id': {'slug': 'a_tus_pies', 'artista': 'marco_barrientos'}, 'autores': ['Marco barrientos', 'Marcos Barrientos,klaus Kuehn.'], 'autor_count': 2}
{'_id': {'slug': 'el_duelo_unplugged', 'artista': 'ley'}, 'autores': ['Bobe, Cuevas', 'la Ley\xa0 '], 'autor_count': 2}
{'_id': {'slug': 'extranio', 'artista': 'rodriguez'}, 'autores': ['los Rodriguez\xa0 ', 'Julian Infante'], 'autor_count': 2}
{'_id': {'slug': 'y_lo_que_quiero_es_que_pises', 'artista': 'catupecu_machu'}, 'autores': ['F. Ruiz Díaz, G. Ruiz Díaz, M. Sosa', 'Fernando ruiz diaz'], 'autor_count': 2}
{'_id': {'slug': 'lamina', 'artista': 'flema'}, 'autores': ['Ricky Espinosa', 'Ricky'], 'autor_count': 2}
{'_id': {'slug': 'ahi_ahi', 'artista': 'silvestre_dangon'}, 'autores': ['Jorge Valbuena', 'Silvestre Dangond\xa0 '], 'autor_count': 2}
{'_id': {'slug': 'por_que', 'artista': 'floricienta_y_su_banda'}, 'autores': ['Floricienta y su banda\xa0 ', 'María Cristina De Giácomi, Carlos Nilson'], 'autor_count': 2}
{'_id': {'slug': 'que_m

El artista también depende de la versión

### Búsqueda de texto

In [27]:
query = '"buena suerte y hasta luego"'
fields = {"versiones.acordes":0, "versiones.contenido": 0}
cur = songs.find({"$text": {"$search": query}},
                 {"versiones.acordes":0, "versiones.contenido": 0, "score": {"$meta": "textScore"}}).sort(
    [('score', {"$meta": "textScore"})]).limit(5)
show_cursor(cur)

{'_id': 'andres_calamaro/buena_suerte_y_hasta_luego',
 'album': 'Grabaciones Encontradas (Dos)',
 'artista': 'andres_calamaro',
 'nombre': 'Buena Suerte y hasta luego',
 'nombre_artista': 'Andrés Calamaro',
 'score': 3.0,
 'slug': 'buena_suerte_y_hasta_luego',
 'versiones': [{'_id': 'acal0722',
                'album': 'Grabaciones Encontradas (Dos)',
                'autor': 'Andrés Calamaro',
                'formato': 'R',
                'puntaje': 8.01,
                'transcriptor': 'roman_tico87',
                'votos': 91},
               {'_id': 'acal0723',
                'album': 'Grabaciones Encontradas (Dos)',
                'autor': 'Andrés Calamaro',
                'formato': 'R',
                'puntaje': 8.02,
                'transcriptor': '14946',
                'votos': 69},
               {'_id': 'acal0721',
                'album': 'Grabaciones Encontradas (Dos)',
                'autor': 'Andrés Calamaro',
                'formato': 'R',
                '